In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ls "/content/drive/My Drive/Data Alica method less noisy new/Image 1/"

!for i in {1..10}; do cp "/content/drive/My Drive/Data Alica method less noisy new/Image $i/cropped_noisy_compilation_$i.npy" "Noi$i.npy"; done
!for i in {1..10}; do cp "/content/drive/My Drive/Data Alica method less noisy new/Image $i/cropped_original_compilation_$i.npy" "Ori$i.npy"; done
!for i in {1..10}; do cp "/content/drive/My Drive/Data Alica method less noisy new/Image $i/similarity_rating_compilation_$i.npy" "Label$i.npy"; done

In [0]:
import os
import torch
import torchvision
import torch.utils.data as utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import timeit
import random

import matplotlib.pyplot as plt
import numpy as np

def imageBatch(nb_image):
    imgBatchR_T = torch.randint(0,255,(nb_image,3,9,9))
    imgBatchL_T = imgBatchR_T
    labelT = torch.ones(nb_image,1)

    imgBatchR_F = torch.randint(0,255,(nb_image,3,9,9)) #
    imgBatchL_F = torch.randint(0,255,(nb_image,3,9,9))
    labelF = torch.zeros(nb_image,1)

    finalR = torch.cat((imgBatchR_T,imgBatchR_F))
    finalL = torch.cat((imgBatchL_T,imgBatchL_F))
    finalLabel = torch.cat((labelT,labelF))

    return finalR, finalL, finalLabel

class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

def firstStageCNN():
    return nn.Sequential(nn.Linear(3*9*9, 50), #L1
                         nn.ReLU(inplace=True),
                         
                         nn.Linear(50, 50), #L2
                         nn.ReLU(inplace=True))

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.cnn1 = firstStageCNN()
        
        self.cnn2 = firstStageCNN()

        
        self.fc = nn.Sequential(nn.Linear(100, 75), #L3
                                nn.ReLU(inplace=True),
                                
                                nn.Linear(75, 50), #L4
                                nn.ReLU(inplace=True),
                            
                                nn.Linear(50, 25), #L5
                                nn.ReLU(inplace=True),
                                
                                nn.Linear(25, 1), #L6
                                nn.Sigmoid())
    
    def forward(self, input1, input2):
        
        output1 = self.cnn1(input1.float().view(-1,3*9*9))
        output2 = self.cnn2(input2.float().view(-1,3*9*9))


        combined = torch.cat((output1.view(output1.size(0), -1),
                              output2.view(output2.size(0), -1)), dim=1)

        combined = torch.unsqueeze(combined,2)
        combined = torch.unsqueeze(combined,3)
        combined = combined.view(-1,100)
        
        out = self.fc(combined)
        
        return out

def train(net, finalR, finalL, finalLabel, EPOCHS, BATCH_SIZE):
    height, width = 9,9
    optimizer = optim.Adam(net.parameters(), lr=0.0005)
    loss_function = nn.BCELoss()
    dataset = utils.TensorDataset(finalL, finalR, finalLabel)
    train_dataloader = DataLoader(dataset, shuffle=True, num_workers=8, batch_size=BATCH_SIZE)
    COUNTER = 1
    net.zero_grad()

    print("train function was executed")
    for epoch in range(EPOCHS):
        avg_loss = 0
        for i, data in enumerate(train_dataloader):
            img1, img2, depthmap = data
            # img1, img2, depthmap = img1.cuda(), img2.cuda(), depthmap.cuda()
            optimizer.zero_grad() # reset gradient
            outputs = net(img1, img2)
            outputs = outputs.cpu()
            depthmap = depthmap.cpu()
            loss = loss_function(outputs, depthmap)
            #print("Loss:", float(loss))
            avg_loss+=loss
            loss.backward()
            optimizer.step()

        #Print out images and epoch numbers 
        print("Epoch number: ", COUNTER)
        COUNTER += 1 
        print("Average Loss:", avg_loss/i)
        outputs = outputs.cpu()
        # img1 = img1.cpu()
        # img2 = img2.cpu()
        depthmap = depthmap.cpu()

        # image = np.swapaxes(img1.numpy(), 1,3)
        # image1 = np.swapaxes(img2.numpy(), 1,3)
        # plt.figure()
        # plt.imshow(image[0].astype('uint8'))
        # plt.figure()
        # plt.imshow(image1[0].astype('uint8'))
        # plt.show()
        print('Likeliness value:', outputs[-1])
        print("Actual Likeness", depthmap[-1])
    return net

In [0]:
NOISY, ORIG, LABEL = [],[],[] 
for i in range(10):
  NOISY.append(np.load("Noi"+str(i+1)+".npy"))
  ORIG.append(np.load("Ori"+str(i+1)+".npy"))
  LABEL.append(np.load("Label"+str(i+1)+".npy"))

NOISY = np.concatenate(tuple(NOISY))
ORIG = np.concatenate(tuple(ORIG))
LABEL = np.concatenate(tuple(LABEL)) 

In [0]:
NOISY = torch.from_numpy(NOISY).float()
ORIG = torch.from_numpy(ORIG).float()
LABEL = torch.from_numpy(LABEL).float()

net = SiameseNetwork()
batch_size = 10
EPOCHS = 500

In [0]:
final = train(net,ORIG,NOISY,LABEL,EPOCHS,batch_size)

In [0]:
# Saving the model
from google.colab import files
torch.save(final, "disparity_NN.pth")
files.download("disparity_NN.pth")